In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os


**Data Visualization**

Visualizing Data is a very important step in a Data Science project.  As per the recent Kagagle Survey 10-20% of the total Data Science Project time is spent on visualizing data. https://www.kaggle.com/rajeshcv/state-of-data-science-machine-learning-2018

SaS Data Visualization’s webpage explain Data visualization  beautifully.

*'The way the human brain processes information, using charts or graphs to visualize large amounts of complex data is easier than poring over spreadsheets or reports. Data visualization is a quick, easy way to convey concepts in a universal manner — and you can experiment with different scenarios by making slight adjustments.'*

In the Santander Customer Transaction Prediction competition the features are predominently numeric.

This kernel's objective is to
*     Understand the value distribution in various features through boxplots and histograms. 
*     Seggregate features into groups based on range of values.
*     Identify  features with similiar value distribution.
*     To understand if there is any difference in values between the two target groups 'transaction done and 'transaction not done'
*  Check whether feature values in test and train comes from the same sampling  distribution.




In [2]:
train = pd.read_csv('./feature/101_train.csv')
test = pd.read_csv('./feature/101_test.csv')

In [3]:
train.head()

,Unnamed: 0,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 203 entries, Unnamed: 0 to var_199
dtypes: float64(200), int64(2), object(1)
memory usage: 309.8+ MB


All the 200 features have numeric values . Let's check first if some of these numeric features are categorical  or boolean .In that case they will have less than 500 unique values.

In [5]:
likely_cat = {}
for c in train.columns:
    likely_cat[c] = 1.*train[c].nunique()/train[c].count() < 0.005
likely_cat= pd.Series(likely_cat)
likely_cat[likely_cat==True]

target    True
var_68    True
dtype: bool

In [6]:
train.var_68.nunique()

451

None of the features are categorical or boolean except 'target'. Let's understand the range of values of each of these features by plotting the max, min and median value of the features.

In [7]:
trainvaluedist = pd.DataFrame(train.iloc[:,2:].max(axis=0),columns=["Max_value"])
trainvaluedist['Min_value'] = train.iloc[:,2:].min(axis=0)
trainvaluedist['Median_value'] = train.iloc[:,2:].median(axis=0)
trainvaluedist.head()

,Max_value,Min_value,Median_value
target,1.0000,0.0000,0.00000
var_0,20.3150,0.4084,10.52475
var_1,10.3768,-15.0434,-1.60805
var_2,19.3530,2.1171,10.58000
var_3,13.1883,-0.0402,6.82500


In [8]:
sns.set(rc={'figure.figsize':(24,12)})
line=sns.lineplot(data=trainvaluedist )
line= line.set(yticks=[-80,-60,-40,-30,-20,-10,0,10,20,30,40,60,80])


AttributeError: module 'seaborn' has no attribute 'lineplot'

Combined all values in the features are between 80 and -90.  From the plot it looks like the features can be seggregrated into 10 groups  based on their max and min values.

In [9]:
colzerototen= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].min() >=0) & (train.loc[:,c].max()< 10) ]
print('Number of features with positive values and maximum value less than 10 :',len(colzerototen))
colzerototwenty= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].min() >=0) & (train.loc[:,c].max() >= 10) & (train.loc[:,c].max() < 20)  ]
print('Number of features with positive values maximum value between 10 & 20 :',len(colzerototwenty))
colzeroandtwentyplus= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].min() >=0) & (train.loc[:,c].max() >= 20)]
print('Number of features with positive values maximum value > 20 :',len(colzeroandtwentyplus))
colzerominus= [c for c in train.iloc[:,2:].columns if train.loc[:,c].max() <0 ]
print('Number of features with only negative values :',len(colzerominus))
colplustenminusten= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].max() <= 10) & (train.loc[:,c].min() >=-10 )& (train.loc[:,c].min()< 0 )]
print('Number of features with negative values between 10 and -10 :',len(colplustenminusten))
colplustwentyminusten= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].max() <= 20)& (train.loc[:,c].max() > 10) & (train.loc[:,c].min() >=-10 ) & (train.loc[:,c].min() < 0 )]
print('Number of features with max value between 10 and 20 and min value between  between 0 and -10  :',len(colplustwentyminusten))
colplustwentyminustwenty= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].max() <= 20) &  (train.loc[:,c].min() < -10 ) & (train.loc[:,c].min() >= -20 )]
print('Number of features with max value less than 20 and min value between -10 and -20 :',len(colplustwentyminustwenty))
colplustwentyminustwentyless= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].max() <= 20)& (train.loc[:,c].min() < -20 )]
print('Number of features with max value less than 20 and min value less than -20 :',len(colplustwentyminustwentyless))
colplustwentymoreminusten= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].max() >20)& (train.loc[:,c].min()< 0 ) & (train.loc[:,c].min()>= -10 )]
print('Number of features with max value more than 20 and min value more than -10 :',len(colplustwentymoreminusten))
colplustwentymoreminustwenty= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].max() >20)& (train.loc[:,c].min()< -10 ) & (train.loc[:,c].min()>= -20 )]
print('Number of features with max value more than 20 and min value between -10 and -20:',len(colplustwentymoreminustwenty))
colplustwentymoreminustwentymore= [c for c in train.iloc[:,2:].columns if (train.loc[:,c].max() >20)& (train.loc[:,c].min()< -20 )]
print('Number of features with max value more than 20 and min value less than -20:',len(colplustwentymoreminustwentymore))

Number of features with positive values and maximum value less than 10 : 17
Number of features with positive values maximum value between 10 & 20 : 31
Number of features with positive values maximum value > 20 : 22
Number of features with only negative values : 0
Number of features with negative values between 10 and -10 : 14
Number of features with max value between 10 and 20 and min value between  between 0 and -10  : 17
Number of features with max value less than 20 and min value between -10 and -20 : 15
Number of features with max value less than 20 and min value less than -20 : 16
Number of features with max value more than 20 and min value more than -10 : 28
Number of features with max value more than 20 and min value between -10 and -20: 17
Number of features with max value more than 20 and min value less than -20: 24


**Features with  positive values and maximum value less than 10**

In [10]:
sns.set(rc={'figure.figsize':(20,8)})
setpositive=train.loc[:,colzerototen].boxplot(rot=90)
setpositive=setpositive.set(yticks=[0,2.5,5,7.5,10],title="Features with  positive values and maximum value less than 10")

var_68,var_91,var_103,var_148 and var_161 have comparatively lower range of values .
The histograms below shows the distribution of values in cases of transaction done in green color (target=1) and transaction not done (target=0) in red colour.

In [11]:
sns.set(rc={'figure.figsize':(20,16)})
plotlist =['hist'+ str(col) for col in colzerototen]

for k in range(len(colzerototen)):
     plt.subplot(4,4,k+1)
     plotlist[k] =plt.hist(train[colzerototen[k]])
     #plotlist[k].set(title=colzerototen[k])
    


ValueError: num must be 1 <= num <= 16, not 17

In [ ]:
sns.set(rc={'figure.figsize':(20,16)})
def sephist(col):
    yes = train[train['target'] == 1][col]
    no = train[train['target'] == 0][col]
    return yes, no

for num, alpha in enumerate(colzerototen):
    plt.subplot(4, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

var_103 values lie  between 1.1 and 2 , var_148 between 3.4 and 4.6 , var_68 values are in a narrow range between 4.99 and 5.04,   var_161 between 5 and 6.2  &  var_91 between 6.6 and 7.4.  Considering var_166 with values between 2 and 4 and var_169 and var_133  they all appear to be in sequence.

However there is no significant difference in values between the "transaction done" and "transaction not done" groups

**Features with  positive values and maximum value between 10 & 20**

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
setpositive20=train.loc[:,colzerototwenty].boxplot(rot=90)
setpositive20=setpositive20.set(yticks=[0,5,10,15,20],title="Features with  positive values and maximum value between 10 & 20")

var_12,  var_15 ,var_25, var_34,  var_43, var_108, var_125 have very low range of values further elaborated by the histogram below.

In [ ]:
sns.set(rc={'figure.figsize':(16,24)})
for num, alpha in enumerate(colzerototwenty):
    plt.subplot(8, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

All those variables with a short range of values have values in the range 10 to 15 and as in the earlier group appear to be in some sequence.

**Features with  positive values and maximum value greater than 20**

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
setpositive20plus=train.loc[:,colzeroandtwentyplus].boxplot(rot=90)
setpositive20plus=setpositive20plus.set(yticks=[0,10,20,30,40],title="Features with  positive values and maximum value more than 20")

In [ ]:
sns.set(rc={'figure.figsize':(16,20)})
for num, alpha in enumerate(colzeroandtwentyplus):
    plt.subplot(6, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

var_85, var_194 and  var_198 appear to have similiar distribution of values.  var_0, var_46 , var_56, var_175 and var_177 also appear to have a similiar value distribution.

**Features with  values between 10 and -10**

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
setplustenminusten = train.loc[:,colplustenminusten].boxplot(rot=90)
setplustenminusten = setplustenminusten.set(yticks=[-10,-5,0,5,10],title="Features with  values between 10 and -10")

In [ ]:
sns.set(rc={'figure.figsize':(16,16)})
for num, alpha in enumerate(colplustenminusten):
    plt.subplot(4, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

**Features with  max value between 10 &  20  and min values between 0 & -10**

In [ ]:
sns.set(rc={'figure.figsize':(16,8)})
setplustwentyminusten = train.loc[:,colplustwentyminusten].boxplot(rot=90)
setplustwentyminusten = setplustwentyminusten.set(yticks=[-10,-5,0,5,10,15,20],title="Features with  max value between 10 and 20 and min values between 0 and -10")

In [ ]:
sns.set(rc={'figure.figsize':(16,16)})
for num, alpha in enumerate(colplustwentyminusten):
    plt.subplot(5, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

From the above histogram for many of the features the ' transaction done'  group in green seems to have  lower range than the 'transaction not done' group in red.

**Features with  max value between 10 &  20  and min values between -10 & -20**

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
setplustwentyminustwenty=train.loc[:,colplustwentyminustwenty].boxplot(rot=90)
setplustwentyminustwenty=setplustwentyminustwenty.set(yticks=[-20,-15,-10,-5,0,5,10,15,20],title="Features with  max value between 10 and 20 and min values between -10  and -20")

In [ ]:
sns.set(rc={'figure.figsize':(16,16)})
for num, alpha in enumerate(colplustwentyminustwenty):
    plt.subplot(4, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

var_39,var_65 and var_138 appear to have similiar distribution of values and so is var_63 and var_128
For some of the features the ' transaction done'  group in green seems to have  lower range than the 'transaction not done' group in red.

**Features with  max value between 10 &  20  and min values less than  -20**

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
setplustwentyminustwentyless=train.loc[:,colplustwentyminustwentyless].boxplot(rot=90)
setplustwentyminustwentyless=setplustwentyminustwentyless.set(yticks=[-40,-30,-20,-15,-10,-5,0,5,10,15,20],title="Features with  max value between 10 and 20 and min values less than -20")

In [ ]:
sns.set(rc={'figure.figsize':(16,16)})
for num, alpha in enumerate(colplustwentyminustwentyless):
    plt.subplot(4, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

var_84, var _155, var_157 appear to have similiar distribution of values and so does  var_11, var_180 & var_185

**Features with  max value greater than 20  and min values less than  -20**

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
setplustwentymoreminusten=train.loc[:,colplustwentymoreminusten].boxplot(rot=90)
setplustwentymoreminusten=setplustwentymoreminusten.set(yticks=[-10,-5,0,5,10,15,20,30,40,60],title="Features with  max value more than 20 and min values between 0 and -10")

In [ ]:
sns.set(rc={'figure.figsize':(16,16)})
for num, alpha in enumerate(colplustwentymoreminusten):
    plt.subplot(7, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

(var_73 & var_158) , (var_92,var_154,var_159 & var_163) , ( var_20 & var_55) 
The features within the above groups  appear to have similiar distribution of values . Here also the range  of values for the *'transaction done'* group in green appears to be shorter.

**Features with  max value more than 20 and min values between -10 and -20**

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
setplustwentymoreminustwenty=train.loc[:,colplustwentymoreminustwenty].boxplot(rot=90)
setplustwentymoreminustwenty=setplustwentymoreminustwenty.set(yticks=[-20,-15,-10,-5,0,5,10,15,20,30,40,60],title="Features with  max value more than 20 and min values between -10 and -20")

In [ ]:
sns.set(rc={'figure.figsize':(16,16)})
for num, alpha in enumerate(colplustwentymoreminustwenty):
    plt.subplot(7, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

var_21 & var_172 appear to have similiar distribution of values

**Features with  max value more than 20 and min values less than -20**

In [ ]:
sns.set(rc={'figure.figsize':(20,8)})
setplustwentymoreminustwentymore=train.loc[:,colplustwentymoreminustwentymore].boxplot(rot=90)
setplustwentymoreminustwentymore=setplustwentymoreminustwentymore.set(yticks=[-40,-30,-20,-10,0,5,10,15,20,30,40,60],title="Features with  max value more than 20 and min values less than -20")

In [ ]:
sns.set(rc={'figure.figsize':(16,16)})
for num, alpha in enumerate(colplustwentymoreminustwentymore):
    plt.subplot(6, 4, num+1)
    plt.hist(sephist(alpha)[0], alpha=0.75, label='yes', color='g')
    plt.hist(sephist(alpha)[1], alpha=0.25, label='no', color='r')
    plt.legend(loc='upper right')
    plt.title(alpha)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

var_47 & var_187 appear to have similiar distribution of values

**Checking for correlation between feature**s

In [ ]:
traincorr = train.iloc[:,2:].corr()
traincorr.head()

In [ ]:
sns.set(rc={'figure.figsize':(10,8)})
sns.heatmap(traincorr,xticklabels=traincorr.columns,yticklabels=traincorr.columns,cmap=sns.diverging_palette(240, 10, n=9))

***No correlation between any features . Does this mean all features are important?***

**Kolmogorov-Smirnov test**

Before concluding let's do a check of whether feature values in test and train comes from the same sampling  distribution.
Kolmogorov-Smirnov  is a two-sided test for the null hypothesis that 2 independent samples are drawn from the same continuous distribution.
If the K-S statistic is small or the p-value is high, then we cannot reject the hypothesis that the distributions of the two samples are the same.

In [12]:
from scipy.stats import ks_2samp
from tqdm import tqdm
ks_values =[]
p_values  = []
train_columns = train.iloc[:,2:].columns
for i in tqdm(train_columns):
    ks_values.append(ks_2samp(test[i] , train[i])[0])
    p_values.append(ks_2samp(test[i] , train[i])[1])
p_values_series = pd.Series(p_values, index = train_columns) 


  0%|                                                                                          | 0/201 [00:00<?, ?it/s]


KeyError: 'target'

For the two tailed test at 95% confidence level the pvalue has to be less than 0.05 to reject the null hypothesis that both samples are from same distribution.Let's look for values less than 0.05

In [ ]:
dissimiliar_features= p_values_series[p_values_series <0.05].index
dissimiliar_features

As per the Kolmogorov-Smirnov test 46 features have a high probability of not being from the same sampling distribution.
Will this affect the models? 

In [ ]:
# sns.set(rc={'figure.figsize':(16,48)})
# set1=train.iloc[:,47:92].hist(layout=(9,5),sharey=True)